In [1]:
import pandas as pd
import numpy as np

In [2]:
d1c = pd.read_csv("d1/comments.csv")
d1f = pd.read_csv("d1/follows.csv")
d1pt = pd.read_csv("d1/photo_tags.csv")
d1p = pd.read_csv("d1/photos.csv")
d1t = pd.read_csv("d1/tags.csv")
d1u = pd.read_csv("d1/users.csv")

In [3]:
d2 = pd.read_csv("d2/Instagram_Reels_Data_Cleaned.csv").
d3 = pd.read_csv("d3/insta_sentiment_data.csv")
d4 = pd.read_csv("d4/instagram_reach.csv")

In [14]:
photo_tags = pd.merge(
    d1pt, d1t, left_on='tag_id', right_on='id', how='left')

In [11]:
photo_tags.head(4)

,photo_id,tag_id,id,tag_name,created_at
0,14,1,1,sunset,2023-05-13 15:14:58
1,21,1,1,sunset,2023-05-13 15:14:58
2,45,1,1,sunset,2023-05-13 15:14:58
3,75,1,1,sunset,2023-05-13 15:14:58


In [18]:
# Merge with comments dataset to get comments per photo
photos_with_comments = pd.merge(
    d1p,d1c, left_on='id', right_on='photo_id', how='left')
photos_with_comments.head(3)

,id_x,image_url,user_id_x,created_dat,id_y,comment_text,user_id_y,photo_id,created_at
0,1,http://elijah.biz,1,2023-05-13 15:14:58,1.0,unde at dolorem,2.0,1.0,2023-05-13 15:14:58
1,1,http://elijah.biz,1,2023-05-13 15:14:58,2.0,quae ea ducimus,3.0,1.0,2023-05-13 15:14:58
2,1,http://elijah.biz,1,2023-05-13 15:14:58,3.0,alias a voluptatum,5.0,1.0,2023-05-13 15:14:58


In [24]:
photo_tags.shape
photos_with_comments.shape

(1222, 9)

In [ ]:
# Merge with the photo-tags dataset to include tags associated with each photo
photos_tags_comments = pd.merge(
    photos_with_comments, photo_tags, left_on='id', right_on='photo_id', how='left')

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [28]:
# Step 1: Vectorize Tags (TF-IDF)
tag_vectorizer = TfidfVectorizer()
tag_matrix = tag_vectorizer.fit_transform(d1t['tag_name'])

In [29]:
# Step 2: Vectorize Comments (TF-IDF)
comment_vectorizer = TfidfVectorizer()
comment_matrix = comment_vectorizer.fit_transform(d1c['comment_text'])

In [34]:
# Step 3: Vectorize Hashtags (TF-IDF)
d2['hashtags'] = d2['hashtags'].fillna('')
hashtag_vectorizer = TfidfVectorizer()
hashtag_matrix = hashtag_vectorizer.fit_transform(d2['hashtags'])

In [39]:
d1t.head()

,id,tag_name,created_at
0,1,sunset,2023-05-13 15:14:58
1,2,photography,2023-05-13 15:14:58
2,3,sunrise,2023-05-13 15:14:58
3,4,landscape,2023-05-13 15:14:58
4,5,food,2023-05-13 15:14:58


In [55]:
input_photo_id = 1  # Example photo_id

In [56]:
input_tags_vector = tag_matrix[input_photo_id]
input_comments_vector = comment_matrix[input_photo_id]
input_hashtags_vector = hashtag_matrix[input_photo_id]

In [57]:
# Compute similarities for each feature set
tag_similarities = cosine_similarity(input_tags_vector, tag_matrix)
comment_similarities = cosine_similarity(input_comments_vector, comment_matrix)
hashtag_similarities = cosine_similarity(input_hashtags_vector, hashtag_matrix)

In [58]:
# Step 6: Normalize the similarities between 0 and 1 for fair comparison
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [59]:
tag_similarities_scaled = scaler.fit_transform(tag_similarities)
comment_similarities_scaled = scaler.fit_transform(comment_similarities)
hashtag_similarities_scaled = scaler.fit_transform(hashtag_similarities)

In [50]:
# Tags Similarity
tag_similarities = cosine_similarity(
    tag_matrix[input_photo_id], tag_matrix).flatten()  # 1D array

In [51]:
# Comments Similarity
comment_similarities = cosine_similarity(
    comment_matrix[input_photo_id], comment_matrix).flatten()  # 1D array

In [52]:
# Hashtags Similarity
hashtag_similarities = cosine_similarity(
    hashtag_matrix[input_photo_id], hashtag_matrix).flatten()  # 1D array

In [62]:
# Step 7: Get top K recommendations based on combined similarity
top_k_photos = tag_similarities.argsort(
)[::-1][:10]  # Top 10 recommendations
print("Top recommended photos:", top_k_photos)

Top recommended photos: [[ 0 18 17 16 15 14 13 12 11 19 10  8  7  6  5  4  3  2  9 20  1]]
